In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

os.chdir('../')
import data
from transformer_model import MinMax
from losses import stats, MAAPE, wMAPE, sMAPE, RMSE, MAE, summary_erros
os.chdir('notebooks')

2023-01-30 10:36:46.446074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
transactions_path = '../../data/transactions.parquet'
stations_path = '../../data/stations_DB.parquet'
aggregation = "15-mins"
max_stations = None
max_transactions = 10000
train_date = '2015-12-01'

train_data, test_data, metadata = data.tf_data(
        transactions_path,
        stations_path,
        aggregation,
        train_date,
        max_transactions,
        max_stations)

2023-01-30 10:37:14.257367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



Train features shape: (8206, 18, 147)
Test features shape: (730, 18, 147)

Train time_embeddings shape: (8206, 19, 8)
Test time_embeddings shape: (730, 19, 8)

Train spatial_embeddings shape: (8206, 147, 2)
Test spatial_embeddings shape: (730, 147, 2)

Train labels shape: (8206, 147)
Test labels shape: (730, 147)

Train status shape: (8206, 19, 147)
Test status shape: (730, 19, 147)



In [ ]:
def model_fit(model, train, test, name = 'default'):
    x_train = train[0]
    x_train_status = train[1]
    y_train = train[2]

    x_test = test[0]
    x_test_status = test[1]
    y_test = test[1]
    
    norm = MinMax()
    norm.adapt(x_train)
    
    x_train = norm(x_train)
    y_train = norm(y_train)
    x_test = norm(x_test)
    
    # For compiling and training
    loss_fn = tf.losses.MeanSquaredError()
    optimizer = tf.optimizers.Adam(0.001)
    accuracy_fn = [tf.metrics.MeanAbsoluteError()]
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss',
                                                            patience=5,
                                                            mode='min')
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=[accuracy_fn])
    model.fit(
        x = [x_train, x_train_status],
        y = y_train,
        epochs=100,
        callbacks=[early_stopping],
        batch_size=256, verbose = False)
    
    prediction = model.predict([x_test,x_test_status])
    prediction = norm(prediction, reverse = True)
    prediction = pd.DataFrame(prediction)
    prediction.name = name
    return prediction

## LSTM Model

In [ ]:
class LSTM(tf.keras.Model):
    def __init__(self, lstm_units = 32, out_dimension =147 ):
        super().__init__()
        
        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=False)
        self.dense = tf.keras.layers.Dense(units = out_dimension, activation = 'linear')
        self.clousures = tf.keras.layers.Multiply()

    def call(self, inputs):
        x, clousures = inputs
        
        x = self.lstm(x)
        x = self.dense(x)
        x = self.clousures([x, clousures])
        return x 

In [ ]:
lstm_model = LSTM()
x_train = train_data['features'][:,-6:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-6:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_1 = model_fit(lstm_model,
                  [x_train, x_train_status, y_train],
                  [x_test, x_test_status, y_test], 
                  name = 'LSTM-N')

In [ ]:
lstm_model = LSTM()
x_train = train_data['features'][:,-14:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-14:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_2 = model_fit(lstm_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'LSTM-N+')

In [ ]:
lstm_model = LSTM()
x_train = train_data['features'][:,-16:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-16:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_3 = model_fit(lstm_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'LSTM-NC')

In [ ]:
lstm_model = LSTM()
x_train = train_data['features']#[:,-16:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features']#[:,-16:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_4 = model_fit(lstm_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'LSTM-NCT')

In [ ]:
lstm_model = LSTM()
x_train = train_data['features']#[:,-16:,:]
x_train_status = train_data['status']
y_train = train_data['labels']

x_test = test_data['features']#[:,-16:,:]
x_test_status = test_data['status']
y_test = test_data['labels']
exp_5 = model_fit(lstm_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'LSTM-Clousure')

## Dense Model

In [ ]:
class FNN(tf.keras.Model):
    def __init__(self ):
        super().__init__()
        
        self.seq_dense = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 147)
        ])
        self.clousures = tf.keras.layers.Multiply()

    def call(self, inputs):
        x, clousures = inputs
        x = self.seq_dense(x)
        x = self.clousures([x, clousures])
        return x

In [ ]:
dense_model = FNN()
x_train = train_data['features'][:,-6:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-6:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_6 = model_fit(dense_model, 
                 [x_train, x_train_status, y_train],
                 [x_test, x_test_status, y_test], name = 'FNN-N')

In [ ]:
dense_model = FNN()
x_train = train_data['features'][:,-14:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-14:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_7 = model_fit(dense_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'FNN-N+')

In [ ]:
dense_model = FNN()
x_train = train_data['features'][:,-16:,:]
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features'][:,-16:,:]
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_8 = model_fit(dense_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'FNN-NC')

In [ ]:
dense_model = FNN()
x_train = train_data['features']
x_train_status = tf.ones_like(train_data['status'])
y_train = train_data['labels']

x_test = test_data['features']
x_test_status = tf.ones_like(test_data['status'])
y_test = test_data['labels']
exp_9 = model_fit(dense_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'FNN-NCT')

In [ ]:
dense_model = FNN()
x_train = train_data['features']#[:,-16:,:]
x_train_status = train_data['status']
y_train = train_data['labels']

x_test = test_data['features']#[:,-16:,:]
x_test_status = test_data['status']
y_test = test_data['labels']
exp_10 = model_fit(dense_model,
                  [x_train, x_train_status, y_train], 
                  [x_test, x_test_status, y_test], 
                  name = 'FNN-Clousure')

## CNN

In [ ]:
# The inputs are 128-length vectors with 10 timesteps, and the
# batch size is 4.
input_shape = (32, 18, 147)
x = tf.random.normal(input_shape)
CON = tf.keras.layers.Conv1D(
147, 3, activation='relu',input_shape=input_shape[1:])
y = CON(x)
print(y.shape)

In [ ]:
CON.weights

In [ ]:
class CNN(tf.keras.Model):
    def __init__(self ):
        super().__init__()
        
        self.seq_dense = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 735),
            tf.keras.layers.Dense(units = 147)
        ])
        self.clousures = tf.keras.layers.Multiply()

    def call(self, inputs):
        x, clousures = inputs
        x = self.seq_dense(x)
        x = self.clousures([x, clousures])
        return x

In [ ]:
stations = metadata['list_stations']

In [ ]:
index = metadata['test_date_index']

In [ ]:
index

In [ ]:
exp_1.index = metadata['test_date_index']
exp_1.columns = metadata['list_stations']
filter_time = (exp_1.index.hour > 5) & (exp_1.index.hour < 22) & (exp_1.index.weekday <5)

In [ ]:
exp_1[filter_time]

In [ ]:
summary_erros(y_test.numpy(), [exp_1, exp_2, exp_3, exp_4, exp_5, 
                               exp_6, exp_7, exp_8, exp_9, exp_10,])